Here we'll show some of the inner workings of the algorithm to compute convex hulls.
First, we'll generate a random set of points as our input data using the random number generation routines in numpy.
To make sure that this demo gives the same results every time, we'll explicitly seed the RNG with the number 1729, which as we all know is [a rather dull one](https://en.wikipedia.org/wiki/1729_(number)).

In [ ]:
import numpy as np
rng = np.random.default_rng(seed=1729)
num_points = 120
X = rng.normal(size=(num_points, 2))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect("equal")
axes.scatter(X[:, 0], X[:, 1]);

To start calculating the convex hull, we'll create a state machine object which we'll call `hull_machine`.
This state machine stores two pieces of data:
1. the current value of the hull geometry in the member `geometry`
2. the *visibility graph* in the member `visible`.

The visibility graph is a weighted bipartite graph between the vertices and the faces of the hull.
The weighting factor between a vertex $v$ and an edge $e$ is the signed area of the triangle formed by $v$ and $e$.

In [ ]:
import zmsh
hull_machine = zmsh.ConvexHullMachine(X)
geometry = hull_machine.geometry

The two methods of the hull machine that we care about are `step` and `is_done`.
The `step` method will find whichever edge of the hull can see the greatest number of points, and then split it along whichever visible point is most extreme.
Any points inside the triangle formed by the old edge and the two new edges will be filtered out as candidate hull points.

To see how this works, we'll step through the hull machine until it's complete.
At every iteration, we'll copy the current value of the topology and the visibility graph.

In [ ]:
from copy import deepcopy
geometries = [deepcopy(hull_machine.geometry)]
visible_cells_ids = []

while not hull_machine.is_done():
    visible_cells_ids.append(hull_machine.get_next_vertex_and_cells()[1])
    hull_machine.step()
    geometries.append(deepcopy(hull_machine.geometry))
    
visible_cells_ids.append([])

Now we'll visualize the state of the algorithm at every step.
The orange edge is the next one that we'll split.

In [ ]:
from ipywidgets import interact
@interact(step=(0, len(geometries) - 1))
def f(step=0):
    geometry = geometries[step]

    fig, ax = plt.subplots()
    ax.set_aspect("equal")

    zmsh.visualize(geometry, dimension=0, ax=ax)

    colors = len(geometry.topology.cells(1)) * ["tab:blue"]
    for cell_id in visible_cells_ids[step]:
        colors[cell_id] = "tab:orange"
    zmsh.visualize(geometry, dimension=1, colors=colors, ax=ax)

We used a random point set for demonstrative purposes here.
Randomized testing is an extraordinarily useful tool, but computational geometry is full of really dreadful edge cases.
For example, what happens if there are three collinear points on the convex hull of a point set?
The middle point isn't necessary to describe the hull; should we include it or not?
The algorithm we used here doesn't include these extraneous collinear points.
But generating three collinear points at random using 64-bit floating point arithmetic is so unlikely that it's practically impossible.
So a naive randomized test suite would be unlikely to find this edge case and the test suite for zmsh explicitly checks for it.